# Laser Circles 

# Code:

In [1]:
import GCode
import GRBL
import numpy as np
from uuid import uuid4
import os
import sys
from time import sleep

sys.path.append("..")
from utils import picture


In [2]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
sleep(0.5)
cnc.laser_mode = 1
sleep(0.5)
print("Laser Mode: {}".format(cnc.laser_mode))



Laser Mode: None


In [3]:
def init(feed=200):
    program = GCode.GCode()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0, Z=0)  # Zero position.
    program.G1(F=feed)
    return program


def end():
    program = GCode.GCode()
    program.G0(X=0, Y=0, Z=0)
    program.M5()
    return program


In [4]:
def circle(center_x=5, center_y=5, radius=5, laser_pwm=255):
    prog = GCode.GCode()
    prog.G0(X=center_x, Y=center_y)
    prog.M3(S=laser_pwm)
    prog.G2(X=center_x - radius, Y=center_y, I=radius, J=0)
    prog.M5()
    prog.G0(X=center_x, Y=center_y)
    return prog


In [5]:
radius = 15


In [6]:
Y_centers = np.linspace(radius, 90 - radius, 4)
X_centers = np.linspace(radius, 180 - radius, 5)


In [7]:
test_run = GCode.GCode()
test_run += init()
for x_center in X_centers:
    for y_center in Y_centers:
        test_run += circle(
            center_x=x_center, center_y=y_center, radius=radius, laser_pwm=5
        )
test_run += end()


In [13]:
gcode_file = "CircleTests.gcode"


In [14]:
test_run.save(gcode_file)

del test_run
test_run = GCode.GCode()

test_run.load(gcode_file)


In [15]:
test_run.buffer[0:5]


['G21', 'G90', 'G92 Y0 X0 Z0', 'G1 F200', 'G0 Y15.0 X15.0']

In [12]:
cnc.run(["G0", "M3S1", "G1F1"])


0.310122013092041

In [ ]:
from time import sleep


In [16]:
test_run


<GCode>[cmds=106]

In [17]:
try:
    cnc.run(test_run)
    while 1:
        print(cnc.status)
        sleep(5)
except KeyboardInterrupt as error:
    print("Feed Hold")
    cnc.cmd("!")
    while 1:
        try:
            cnc.reset()
            break
        except:
            pass
    print("^C")


^C


AssertionError: 

In [ ]:
cnc.cmd("G0X0Y0")


In [ ]:
cnc.reset()


In [ ]:
cnc.reset()


In [ ]:
picture()


# Test Aborted.

Cuts were way too aggressive.